[![](https://img.youtube.com/vi/FnV0thLS1Fs/0.jpg)](https://www.youtube.com/watch?v=FnV0thLS1Fs)

**pixyz  
last update 2022 03 06  
ゆっくりしていってね！**

**こっちも観てね!**

**MMLM Men's EDA ゆっくり実況 日本語＆English** 

https://www.kaggle.com/lunapandachan/mmlm-men-s-english


**[日本語&ENG] MMLM-Men's LGBM baseline ゆっくり実況**  

https://www.kaggle.com/pixyz0130/eng-mmlm-men-s-lgbm-baseline


**参考にしたコード**

**RISHIRAJ ACHARYA : [MMLM22 Men] Tuned 𝘼𝙪𝙩𝙤𝙇𝙂𝘽𝙈 🤯**

https://www.kaggle.com/rishirajacharya/mmlm22-men-s-tuned

# Config

In [89]:
FOR_STAGE2 = True
BASE_PATH = "./data/MDataFiles_Stage2/" if FOR_STAGE2 else "./data/MDataFiles_Stage1/"
RESULTS_NAME = "MNCAATourneyDetailedResults.csv"
SEED_NAME = "MNCAATourneySeeds.csv"
RANK_NAME = "MMasseyOrdinals_thruDay128.csv" if FOR_STAGE2 else "MMasseyOrdinals.csv"
TEST_NAME = "MSampleSubmissionStage2.csv" if FOR_STAGE2 else "MSampleSubmissionStage1.csv"

model_type = 'Autolgbm'  
message='baseline'

train_filename = "./train_#13-book2_stg2.csv" if FOR_STAGE2 else "./train_#13-book2.csv"
output = "./output/#13-book2_stg2" if FOR_STAGE2 else "./output/#13-book2"
test_filename = "./test_#13-book2_stg2.csv" if FOR_STAGE2 else "./test_#13-book2.csv"
task = None
targets = ['target']
features = None
categorical_features = None
use_gpu = False
num_folds = 10
seed = 42
num_trials = 10000
time_limit = 7200
fast = False

TEST = False

In [90]:
if TEST:
    time_limit = 1000

**魔理沙:今回はFOLD5、EPOCH20000で学習をしていくぜ**  

**Marisa: This time, I'll study with FOLD5 and EPOCH20000**

# Import

In [91]:
# !pip install autolgbm

In [92]:
import pandas as pd
import numpy as np
import random
import time
import gc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
import matplotlib.pylab as plt
import seaborn as sns
from autolgbm import AutoLGBM
import pickle

# Overview

**霊夢：今回は回帰タスクのコンペかな？**

**魔理沙：まずは概要を読んでみよう。**

<br>

**Reimu: Is this a regression task competition?**

**Marisa: First, let's read the overview.**

別の年、動揺を予測し、確率を呼び出し、ブラケット学のスキルをリーダーボードテストにかける別のチャンス。3月に開催される第8回機械学習マニア大会では、カグラーズが今年の米国男子大学バスケットボールトーナメントの結果を予測しようとする何百万人ものファンに再び参加します。しかし、ほとんどのファンとは異なり、豊富な履歴データと計算能力の組み合わせを使用して勝者と敗者を選び、テレビでグラウンドトゥルースが展開されます。

過去のNCAAゲームのデータが提供されており、他の公開されているデータソースを使用して勝利を収めることをお勧めします。

この2段階のコンテストの第1段階では、参加者は以前のトーナメントに対してモデルを作成してテストします。第2段階では、参加者は2022年のトーナメントの結果を予測します。2番目のステージに入るのに最初のステージに参加する必要はありません。最初の段階は、モデル構築を奨励し、予測をスコアリングする手段を提供するために存在します。実際の競争は2022年の結果を予測しています。

そして、米国の女子大学バスケットボールトーナメントの結果を予測するために見られる私たちのコンパニオンコンペティションを見るのを忘れないでください！

Another year, another chance to predict the upsets, call the probabilities, and put your bracketology skills to the leaderboard test. In our eighth annual March Machine Learning Mania competition, Kagglers will once again join the millions of fans who attempt to predict the outcomes of this year's US men's college basketball tournament. But unlike most fans, you will pick the winners and losers using a combination of rich historical data and computing power, while the ground truth unfolds on television.

You're provided data of historical NCAA games and are encouraged to use other sources of publicly available data to gain a winning edge.

In stage one of this two-stage competition, participants will build and test their models against previous tournaments. In the second stage, participants will predict the outcome of the 2022 tournament. You don’t need to participate in the first stage to enter the second. The first stage exists to incentivize model building and provide a means to score predictions. The real competition is forecasting the 2022 results.

And don't forget to take a look at our companion competition that looks to predict the outcome of the US women's college basketball tournament!

<img src = "https://storage.googleapis.com/kaggle-competitions/kaggle/4862/media/bball-logo.png" width = 700>

**魔理沙:今回はAutoLGBMを使ったbaselineを作っていくよ。**  

**霊夢：今回はデータの種類がとても多いね。あれ？trainデータはどこ？**

**魔理沙:trainデータは自分でいくつかのデータを組み合わせて作るしかないみたいだな。これは骨が折れそうだ。**

<br>

**Marisa: This time I'm going to make a baseline using lightgbm.**

**Reimu: There are so many types of data this time. that? Where is the train data?**

**Marisa: It seems that train data can only be created by combining some data. This is going to be a pain.**

<img src = "https://4.bp.blogspot.com/-uoVuBWIbdiA/WvQHqpx_YCI/AAAAAAABL8g/NiFZ6K71VBc_0_dcKb3_4nhnvFJ_JMNuACLcBGAs/s450/network_dennou_sekai_figure.png" width = 200>

# Data

**魔理沙:データは以下の通りだぜ。**  

**Marisa: The data is as follows.**

In [93]:
!ls ../input/mens-march-mania-2022/MDataFiles_Stage1

ls: cannot access '../input/mens-march-mania-2022/MDataFiles_Stage1': No such file or directory


**霊夢:一番大事なデータはどれだろう？**

**魔理沙:まずはMNCAATourneyDetailedResultsかな。これを主軸にやっていくぜ。**

**Reimu: Which is the most important data?**

**Marisa: First of all, MNCAA Tourney Detailed Results. Let's do this as the main axis.**

In [94]:
df_TDresults = pd.read_csv(BASE_PATH + RESULTS_NAME)
df_RSDresults = pd.read_csv(BASE_PATH+"MRegularSeasonDetailedResults.csv")
if df_RSDresults is not None:
    df_TDresults = pd.merge(df_RSDresults,df_TDresults,on = df_TDresults.columns.tolist(),how = "outer")
print(df_TDresults.shape)
df_TDresults.head()

(1181, 34)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


In [95]:
# df_TDresults = pd.merge(df_RSDresults,df_TDresults,on = df_TDresults.columns.tolist(),how = "outer")
# print(df_TDresults.shape)

(103213, 34)


**霊夢:FGMとかFGAって何のこと？**

**魔理沙:バスケには詳しくないから具体的には分からないが、どうやらどれもゲーム内で起こした特定のアクションの数みたいだね。**

**霊夢:試合ごとにばらばらだからどうやって扱おうかな？**

**魔理沙:とりあえずSeason、TeamIDごとに平均をとって特徴量にしよう。上手くいくかはちょっとわからないかな。**

<br>

**Reimu: What is FGM or FGA?**

**Marisa: I'm not familiar with basketball, so I don't know the specifics, but it seems that all of them seem to be the number of specific actions taken in the game.**

**Reimu: How do you handle it because it's different for each match?**

**Marisa: For the time being, let's take the average for each Season and Team ID and use it as a feature. I don't know if it will work.**

<img src="http://3.bp.blogspot.com/-KmQQLtEkmLw/U1T3r7D0NdI/AAAAAAAAfVI/c2d4n2kG00U/s400/figure_question.png" width="200">

In [96]:
box_col = ["FGM","FGA","FGM3","FTM","FTA","OR","DR","Ast","TO","Stl","Blk","PF"]
df_boxW = df_TDresults[["Season","WTeamID"]+["W" + col for col in box_col]]
df_boxL = df_TDresults[["Season","LTeamID"]+["L" + col for col in box_col]]
df_boxW = df_boxW.rename(columns={"WTeamID":"TeamID"})
df_boxW = df_boxW.rename(columns={("W"+ col):col for col in box_col})
df_boxL = df_boxL.rename(columns={"LTeamID":"TeamID"})
df_boxL = df_boxL.rename(columns={("L"+ col):col for col in box_col})
df_box = pd.merge(df_boxW,df_boxL,on = ["Season","TeamID"]+box_col,how="outer")
df_box = df_box.groupby(["Season","TeamID"])[box_col].agg(np.mean).reset_index()
df_box.head()

,Season,TeamID,FGM,FGA,FGM3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF
0,2003,1102,19.142857,39.785714,7.821429,11.142857,17.107143,4.178571,16.821429,13.000000,11.428571,5.964286,1.785714,18.750000
1,2003,1103,27.148148,55.851852,5.444444,19.037037,25.851852,9.777778,19.925926,15.222222,12.629630,7.259259,2.333333,19.851852
2,2003,1104,23.965517,57.000000,6.310345,14.793103,20.758621,13.413793,23.793103,12.137931,13.103448,6.448276,3.862069,18.137931
3,2003,1105,24.384615,61.615385,7.576923,15.423077,21.846154,13.500000,23.115385,14.538462,18.653846,9.307692,2.076923,20.230769
4,2003,1106,23.428571,55.285714,6.107143,10.642857,16.464286,12.285714,23.857143,11.678571,17.035714,8.357143,3.142857,18.178571


### WScoreSum, WScoreMean, LScoreSum, LScoreMean

In [97]:
# WinCount
win_df = df_TDresults.groupby(["Season", "WTeamID"], as_index=False)["DayNum"].count()
win_df.rename(columns={"DayNum": "WCount", "WTeamID": "TeamID"}, inplace=True)
# LoseCount
lose_df = df_TDresults.groupby(["Season", "LTeamID"], as_index=False)["DayNum"].count()
lose_df.rename(columns={"DayNum": "LCount", "LTeamID": "TeamID"}, inplace=True)

features_df = pd.concat([win_df[["Season", "TeamID"]], lose_df[["Season", "TeamID"]]], axis=0).drop_duplicates()
features_df = features_df.sort_values(by = ["Season", "TeamID"]).reset_index(drop = True)
features_df = pd.merge(features_df, win_df, on = ["Season", "TeamID"], how = "left")
features_df = pd.merge(features_df, lose_df, on = ["Season", "TeamID"], how = "left")
features_df.fillna(0, inplace = True)

# score sum/mean of Win games in each Season
win_df = df_TDresults.groupby(["Season", "WTeamID"])["WScore"].agg(["sum", "mean"]).reset_index(drop = False)
win_df.rename(columns = {"WTeamID" : "TeamID", "sum" : "WScoreSum", "mean" : "WScoreMean"}, inplace = True)
# score sum/mean of Lose games in each Season
lose_df = df_TDresults.groupby(["Season", "LTeamID"])["LScore"].agg(["sum", "mean"]).reset_index(drop = False)
lose_df.rename(columns = {"LTeamID" : "TeamID", "sum" : "LScoreSum", "mean" : "LScoreMean"}, inplace = True)

features_df = pd.merge(features_df, win_df, on = ["Season", "TeamID"], how = "left")
features_df = pd.merge(features_df, lose_df, on = ["Season", "TeamID"], how = "left")
features_df.fillna(0, inplace = True)





### SeasonScoreSum, SeasonScoreMean

In [98]:
# Score sum/mean in each Season
features_df["SeasonScoreSum"] = features_df["WScoreSum"] + features_df["LScoreSum"]
features_df["SeasonScoreMean"] = features_df["SeasonScoreSum"] / (features_df["WCount"] + features_df["LCount"])

### 

### add Last Season Data

In [99]:
dummy = features_df.copy()
dummy["Season"] = dummy["Season"] + 1
dummy.rename(columns={
    "WCount": "WCount_shift1",
    "LCount": "LCount_shift1",
    "WScoreSum": "WScoreSum_shift1",
    "WScoreMean": "WScoreMean_shift1",
    "LScoreSum": "LScoreSum_shift1",
    "LScoreMean": "LScoreMean_shift1",
    "SeasonScoreSum": "SeasonScoreSum_shift1",
    "SeasonScoreMean": "SeasonScoreMean_shift1",
}, inplace=True)
features_df = pd.merge(features_df, dummy, on=["Season", "TeamID"], how="left")
features_df.fillna(0, inplace=True)

In [100]:
print(list(features_df.columns))
features_df

['Season', 'TeamID', 'WCount', 'LCount', 'WScoreSum', 'WScoreMean', 'LScoreSum', 'LScoreMean', 'SeasonScoreSum', 'SeasonScoreMean', 'WCount_shift1', 'LCount_shift1', 'WScoreSum_shift1', 'WScoreMean_shift1', 'LScoreSum_shift1', 'LScoreMean_shift1', 'SeasonScoreSum_shift1', 'SeasonScoreMean_shift1']


,Season,TeamID,WCount,LCount,WScoreSum,WScoreMean,LScoreSum,LScoreMean,SeasonScoreSum,SeasonScoreMean,WCount_shift1,LCount_shift1,WScoreSum_shift1,WScoreMean_shift1,LScoreSum_shift1,LScoreMean_shift1,SeasonScoreSum_shift1,SeasonScoreMean_shift1
0,2003,1102,12.0,16,825.0,68.750000,778,48.625000,1603.0,57.250000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
1,2003,1103,13.0,14,1141.0,87.769231,986,70.428571,2127.0,78.777778,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
2,2003,1104,17.0,12,1270.0,74.705882,732,61.000000,2002.0,69.034483,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
3,2003,1105,7.0,19,556.0,79.428571,1310,68.947368,1866.0,71.769231,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
4,2003,1106,13.0,15,888.0,68.307692,893,59.533333,1781.0,63.607143,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6887,2022,1468,16.0,13,1185.0,74.062500,786,60.461538,1971.0,67.965517,11.0,7.0,836.0,76.000000,470.0,67.142857,1306.0,72.555556
6888,2022,1469,9.0,18,698.0,77.555556,1140,63.333333,1838.0,68.074074,6.0,13.0,460.0,76.666667,825.0,63.461538,1285.0,67.631579
6889,2022,1470,11.0,17,776.0,70.545455,992,58.352941,1768.0,63.142857,5.0,10.0,345.0,69.000000,613.0,61.300000,958.0,63.866667
6890,2022,1471,11.0,16,870.0,79.090909,976,61.000000,1846.0,68.370370,4.0,10.0,346.0,86.500000,615.0,61.500000,961.0,68.642857


**霊夢:そういえばこのままだと絶対左側のチームIDが勝つことになっちゃうじゃん！**<br>

**魔理沙:そうだった。TeamIDなどのデータを勝ったチームと負けたチームを反転させたデータを下にくっつけよう。これでデータ量も2倍になるぞ。**

<br>

**Reimu: By the way, the team ID on the left side will definitely win!**

**Marisa: That's right. Let's attach the data such as Team ID, which is the reverse of the winning team and the losing team, to the bottom. This will double the amount of data.**

<img src="https://4.bp.blogspot.com/-WOvX4GkiSEo/U1T3wVr3ovI/AAAAAAAAfWg/RpbQYAHDo2Y/s400/figure_shock.png" width = 200>

In [101]:
df_TDresults2 = df_TDresults
df_TDresults = df_TDresults.rename(columns={"WTeamID":"Team1ID","LTeamID":"Team2ID","WScore":"T1Score","LScore":"T2Score"})
df_TDresults = df_TDresults.rename(columns={f"W{col}":f"T1{col}" for col in box_col})
df_TDresults = df_TDresults.rename(columns={f"L{col}":f"T2{col}" for col in box_col})
df_TDresults2 = df_TDresults2.rename(columns={"WTeamID":"Team2ID","LTeamID":"Team1ID","WScore":"T2Score","LScore":"T1Score"})

**霊夢:targetはどうするの？**  

**魔理沙:Team1が勝ったときは1.0、負けたときは0.0にしよう。**

<br>

**Reimu: What about target?**

**Marisa: Let's set it to 1.0 when Team 1 wins and 0.0 when it loses.**

In [102]:
features = ["Season","Team1ID","Team2ID","T1Score","T2Score",'target']
df_TDresults['target'] = 1.0
df_TDresults2['target'] = 0.0
train = pd.merge(df_TDresults,df_TDresults2,on = features,how="outer")
train = train[features]
train.head()

,Season,Team1ID,Team2ID,T1Score,T2Score,target
0,2003,1104,1328,68,62,1.0
1,2003,1272,1393,70,63,1.0
2,2003,1266,1437,73,61,1.0
3,2003,1296,1457,56,50,1.0
4,2003,1400,1208,77,71,1.0


In [103]:
box_T1 = df_box.copy()
box_T2 = df_box.copy()
box_T1.columns = ['Season','Team1ID'] + ["T1"+col+"_mean" for col in box_col]
box_T2.columns = ['Season','Team2ID'] + ["T2"+col+"_mean" for col in box_col]
train = pd.merge(train,box_T1,on = ["Season","Team1ID"],how = "left")
train = pd.merge(train,box_T2,on = ["Season","Team2ID"],how = "left")

**霊夢:今度は何をするの？**

**魔理沙:次はseedを特徴量として組み込もう！**

<br>

**Reimu: What are you going to do next time?**  

**Marisa: Next, let's incorporate seed as a feature!**

<img src = "https://1.bp.blogspot.com/-FrFmmMlJ0bE/U1T3wsYJlRI/AAAAAAAAfWo/7-7pGdnlkKQ/s400/figure_talking.png" width = 200>

In [104]:
df_seeds = pd.read_csv(BASE_PATH + SEED_NAME)
df_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


**霊夢:それぞれのチームのseedが独立した特徴量で正しく学習できるのかな？**

**魔理沙:ちょっと心配だね。seedの差を取った特徴量も作っておこう**

<br>

**Reimu: Can the seeds of each team learn correctly with independent features?**  

**Marisa: I'm a little worried. Let's make a feature quantity with the difference in seed**

In [105]:
df_seeds["seed"] = df_seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds_T1 = df_seeds[['Season','TeamID','seed']].copy()
seeds_T2 = df_seeds[['Season','TeamID','seed']].copy()
seeds_T1.columns = ['Season','Team1ID','T1_seed']
seeds_T2.columns = ['Season','Team2ID','T2_seed']
train = pd.merge(train,seeds_T1,on = ["Season","Team1ID"],how = "left")
train = pd.merge(train,seeds_T2,on = ["Season","Team2ID"],how = "left")
train["seeddiff"] = train["T1_seed"] - train["T2_seed"]

**霊夢:次はどうするの？**

**魔理沙:次はランキングを特徴量にするよ。**  

**霊夢:でもランキングはいろいろ種類があるみたいだけどどうする？**  

**魔理沙：そうだなー、とりあえず平均をとるか。**

**Reimu: What's next?**

**Marisa: Next, let's make the ranking a feature.**

**Reimu: But there seem to be various rankings, but what do you do?**  

**Marisa: That's right, do you take the average for the time being?**

<img src="https://2.bp.blogspot.com/-nMjuko9tXGc/Ut0BV0jJHkI/AAAAAAAAdW0/tRb8t3PEY28/s400/kangaeruhito.png" width = 200>

In [106]:
# df_MMOrdinals = pd.read_csv(BASE_PATH + RANK_NAME) # for Stage 1
# df_MMOrdinals = pd.read_csv("data/MDataFiles_Stage2/MMasseyOrdinals_thruDay128.csv") # for Stage 2
df_MMOrdinals = pd.read_csv(BASE_PATH + RANK_NAME)
df_rank = df_MMOrdinals.groupby(["Season","TeamID"])["OrdinalRank"].agg(np.mean).reset_index()
df_rank.head()

,Season,TeamID,OrdinalRank
0,2003,1102,144.287500
1,2003,1103,182.205000
2,2003,1104,27.655502
3,2003,1105,305.377500
4,2003,1106,243.265000


**魔理沙:こっちもseedと同様に差を取っておこう。**

**Marisa: Let's make a difference here as well as seed.**

In [107]:
ranks_T1 = df_rank.copy()
ranks_T2 = df_rank.copy()
ranks_T1.columns = ['Season','Team1ID','T1_rank_mean']
ranks_T2.columns = ['Season','Team2ID','T2_rank_mean']
train = pd.merge(train,ranks_T1,on = ["Season","Team1ID"],how = "left")
train = pd.merge(train,ranks_T2,on = ["Season","Team2ID"],how = "left")
train["rankdiff"] = train["T1_rank_mean"] - train["T2_rank_mean"]

In [108]:
train = train.drop(columns = ["T1Score","T2Score"])

### merge `features_df`

In [109]:
# preprocess(train)
def preprocess(df):
    df = pd.merge(
        df, features_df, left_on=["Season", "Team1ID"], right_on=["Season", "TeamID"], how="left"
    ).rename(columns = {
            "WCount" : "WCountA",
            "LCount" : "LCountA",
            "WScoreSum" : "WScoreSumA",
            "WScoreMean" : "WScoreMeanA",
            "LScoreSum" : "LScoreSumA",
            "LScoreMean" : "LScoreMeanA",
            "SeasonScoreSum" : "SeasonScoreSumA",
            "SeasonScoreMean" : "SeasonScoreMeanA",

            "WCount_shift1" : "WCount_shift1A",
            "LCount_shift1" : "LCount_shift1A",
            "WScoreSum_shift1" : "WScoreSum_shift1A",
            "WScoreMean_shift1" : "WScoreMean_shift1A",
            "LScoreSum_shift1" : "LScoreSum_shift1A",
            "LScoreMean_shift1" : "LScoreMean_shift1A",
            "SeasonScoreSum_shift1" : "SeasonScoreSum_shift1A",
            "SeasonScoreMean_shift1" : "SeasonScoreMean_shift1A",
    }).drop(columns="TeamID")
    
    df = pd.merge(
        df, features_df, left_on=["Season", "Team2ID"], right_on=["Season", "TeamID"], how="left"
    ).rename(columns={
        "WCount": "WCountB",
        "LCount": "LCountB",
        "WScoreSum": "WScoreSumB",
        "WScoreMean": "WScoreMeanB",
        "LScoreSum": "LScoreSumB",
        "LScoreMean": "LScoreMeanB",
        "SeasonScoreSum": "SeasonScoreSumB",
        "SeasonScoreMean": "SeasonScoreMeanB",

        "WCount_shift1": "WCount_shift1B",
        "LCount_shift1": "LCount_shift1B",
        "WScoreSum_shift1": "WScoreSum_shift1B",
        "WScoreMean_shift1": "WScoreMean_shift1B",
        "LScoreSum_shift1": "LScoreSum_shift1B",
        "LScoreMean_shift1": "LScoreMean_shift1B",
        "SeasonScoreSum_shift1": "SeasonScoreSum_shift1B",
        "SeasonScoreMean_shift1": "SeasonScoreMean_shift1B",
    }).drop(columns="TeamID")
    
    return df


**霊夢:これで一通りのデータができあがったね。**

**魔理沙：でもまだ終わりじゃないよ、testデータも同様にして作っていこう。**

**霊夢：ああ大変だあ**

<br>

**Reimu: That's all the data.**

**Marisa: But it's not over yet, let's make test data in the same way.**

**Reimu: Oh, it's hard**

<img src="https://3.bp.blogspot.com/-3q31Kj8zFHc/V6BP6vEGe1I/AAAAAAAA83s/YD0NZlG5wmgMiB_iX9QMDy8KF2IEid6qQCLcB/s450/figure_korobu.png" width="200">

In [110]:
# test = pd.read_csv(BASE_PATH + TEST_NAME) # for Stage 1
test = pd.read_csv(BASE_PATH + "MSampleSubmissionStage2.csv") # for Stage 2
test.head()

,ID,Pred
0,2022_1103_1104,0.5
1,2022_1103_1112,0.5
2,2022_1103_1116,0.5
3,2022_1103_1120,0.5
4,2022_1103_1124,0.5


In [111]:
test["Season"] = test['ID'].apply(lambda x: int(x[0:4]))
test["Team1ID"] = test['ID'].apply(lambda x: int(x[5:9]))
test["Team2ID"] = test['ID'].apply(lambda x: int(x[10:14]))
test.head()

,ID,Pred,Season,Team1ID,Team2ID
0,2022_1103_1104,0.5,2022,1103,1104
1,2022_1103_1112,0.5,2022,1103,1112
2,2022_1103_1116,0.5,2022,1103,1116
3,2022_1103_1120,0.5,2022,1103,1120
4,2022_1103_1124,0.5,2022,1103,1124


In [112]:
test = pd.merge(test,box_T1,on = ["Season","Team1ID"],how = "left")
test = pd.merge(test,box_T2,on = ["Season","Team2ID"],how = "left")
test.head()

,ID,Pred,Season,Team1ID,Team2ID,T1FGM_mean,T1FGA_mean,T1FGM3_mean,T1FTM_mean,T1FTA_mean,...,T2FGM3_mean,T2FTM_mean,T2FTA_mean,T2OR_mean,T2DR_mean,T2Ast_mean,T2TO_mean,T2Stl_mean,T2Blk_mean,T2PF_mean
0,2022_1103_1104,0.5,2022,1103,1104,23.580645,51.967742,7.709677,14.419355,20.83871,...,9.281250,15.375000,21.062500,11.625000,24.718750,14.625000,14.093750,7.000000,4.718750,18.593750
1,2022_1103_1112,0.5,2022,1103,1112,23.580645,51.967742,7.709677,14.419355,20.83871,...,7.764706,15.911765,21.558824,10.441176,28.352941,19.911765,12.941176,6.705882,5.705882,16.470588
2,2022_1103_1116,0.5,2022,1103,1116,23.580645,51.967742,7.709677,14.419355,20.83871,...,6.363636,17.303030,22.969697,9.696970,25.484848,14.090909,12.151515,7.727273,4.181818,16.848485
3,2022_1103_1120,0.5,2022,1103,1120,23.580645,51.967742,7.709677,14.419355,20.83871,...,8.125000,14.718750,20.093750,10.218750,25.718750,14.531250,11.656250,8.781250,7.843750,18.531250
4,2022_1103_1124,0.5,2022,1103,1124,23.580645,51.967742,7.709677,14.419355,20.83871,...,7.937500,12.125000,17.375000,10.968750,22.750000,15.843750,12.000000,8.812500,3.406250,15.843750


In [113]:
test = pd.merge(test,seeds_T1,on = ["Season","Team1ID"],how = "left")
test = pd.merge(test,seeds_T2,on = ["Season","Team2ID"],how = "left")
test["seeddiff"] = test["T1_seed"] - test["T2_seed"]

In [114]:
test = pd.merge(test,ranks_T1,on = ["Season","Team1ID"],how = "left")
test = pd.merge(test,ranks_T2,on = ["Season","Team2ID"],how = "left")
test["rankdiff"] = test["T1_rank_mean"] - test["T2_rank_mean"]

#### add features

In [115]:
train = preprocess(train)
test = preprocess(test)

#### remove data leak (only for Stage 1)

In [116]:
def concat_row(r):
    if r['Team1ID'] < r['Team2ID']:
        res = str(r['Season'])+"_"+str(r['Team1ID'])+"_"+str(r['Team2ID'])
    else:
        res = str(r['Season'])+"_"+str(r['Team2ID'])+"_"+str(r['Team1ID'])
    return res
def remove_data_leak(df_train, df_test):
    df_train['Concats'] = df_train.apply(concat_row, axis=1)
    df_train_duplicates = df_train[df_train['Concats'].isin(df_test['ID'].unique())]
    df_train_idx = df_train_duplicates.index.values
    print("duplicates: "+len(df_train_idx))
    df_train = df_train.drop(df_train_idx)
    df_train = df_train.drop('Concats', axis=1)

    return df_train

In [ ]:
if not FOR_STAGE2:
    train = remove_data_leak(train, test)

In [117]:
train.head()

,Season,Team1ID,Team2ID,target,T1FGM_mean,T1FGA_mean,T1FGM3_mean,T1FTM_mean,T1FTA_mean,T1OR_mean,...,SeasonScoreSumB,SeasonScoreMeanB,WCount_shift1B,LCount_shift1B,WScoreSum_shift1B,WScoreMean_shift1B,LScoreSum_shift1B,LScoreMean_shift1B,SeasonScoreSum_shift1B,SeasonScoreMean_shift1B
0,2003,1104,1328,1.0,23.965517,57.000000,6.310345,14.793103,20.758621,13.413793,...,2392.0,70.352941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2003,1272,1393,1.0,26.233333,60.300000,7.000000,14.933333,22.833333,14.266667,...,2785.0,79.571429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2003,1266,1437,1.0,27.303030,56.696970,6.121212,17.727273,23.000000,12.939394,...,2166.0,72.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2003,1296,1457,1.0,24.354839,53.064516,6.290323,14.612903,22.387097,13.000000,...,1944.0,69.428571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2003,1400,1208,1.0,27.757576,62.272727,5.939394,17.878788,24.787879,16.090909,...,2138.0,79.185185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
test = test.drop(columns = ['ID','Pred'])
test.head()

,Season,Team1ID,Team2ID,T1FGM_mean,T1FGA_mean,T1FGM3_mean,T1FTM_mean,T1FTA_mean,T1OR_mean,T1DR_mean,...,SeasonScoreSumB,SeasonScoreMeanB,WCount_shift1B,LCount_shift1B,WScoreSum_shift1B,WScoreMean_shift1B,LScoreSum_shift1B,LScoreMean_shift1B,SeasonScoreSum_shift1B,SeasonScoreMean_shift1B
0,2022,1103,1104,23.580645,51.967742,7.709677,14.419355,20.83871,7.870968,23.483871,...,2559.0,79.968750,26.0,7.0,2168.0,83.384615,461.0,65.857143,2629.0,79.666667
1,2022,1103,1112,23.580645,51.967742,7.709677,14.419355,20.83871,7.870968,23.483871,...,2875.0,84.558824,17.0,9.0,1346.0,79.176471,615.0,68.333333,1961.0,75.423077
2,2022,1103,1116,23.580645,51.967742,7.709677,14.419355,20.83871,7.870968,23.483871,...,2539.0,76.939394,25.0,7.0,2107.0,84.280000,497.0,71.000000,2604.0,81.375000
3,2022,1103,1120,23.580645,51.967742,7.709677,14.419355,20.83871,7.870968,23.483871,...,2519.0,78.718750,13.0,14.0,1070.0,82.307692,1014.0,72.428571,2084.0,77.185185
4,2022,1103,1124,23.580645,51.967742,7.709677,14.419355,20.83871,7.870968,23.483871,...,2448.0,76.500000,28.0,2.0,2355.0,84.107143,132.0,66.000000,2487.0,82.900000


In [119]:
print(train.shape)
print(test.shape)

(206426, 66)
(2278, 65)


In [120]:
train.to_csv(train_filename,index = None)
test.to_csv(test_filename,index = None)

In [121]:
train.dtypes

Season                       int64
Team1ID                      int64
Team2ID                      int64
target                     float64
T1FGM_mean                 float64
                            ...   
WScoreMean_shift1B         float64
LScoreSum_shift1B          float64
LScoreMean_shift1B         float64
SeasonScoreSum_shift1B     float64
SeasonScoreMean_shift1B    float64
Length: 66, dtype: object

# Model&Train

**霊夢:やっと学習だ！**

**魔理沙:上手くいくかなあ**

<br>

**Reimu: It's finally learning!**

**Marisa: I wonder if it will work**

<img src="https://1.bp.blogspot.com/-flCQBhvvg6U/V6BP7KK5RdI/AAAAAAAA830/exz5qqb8M0U-hyss5tb4p6St5nF9aq0-wCLcB/s800/figure_ouen.png" width = 200>

In [122]:
features=None

In [123]:
algbm = AutoLGBM(
    train_filename=train_filename,
    output=output,
    test_filename=test_filename,
    task=task,
    targets=targets,
    features=features,
    categorical_features=categorical_features,
    use_gpu=use_gpu,
    num_folds=num_folds,
    seed=seed,
    num_trials=num_trials,
    time_limit=time_limit,
    fast=fast,
)

2022-03-16 00:58:25.835 | INFO     | autolgbm.autolgbm:__post_init__:42 - Output directory: ./output/#13-book2_stg2


In [124]:
algbm.train()
filename = "./output/#13-book2_stg2_model.pickle"
with open(filename, 'bw') as f:
    pickle.dump(algbm, f)

2022-03-16 00:58:39.762 | INFO     | autolgbm.autolgbm:_process_data:149 - Reading training data
2022-03-16 00:58:41.142 | INFO     | autolgbm.utils:reduce_memory_usage:48 - Mem. usage decreased to 25.59 Mb (75.4% reduction)
2022-03-16 00:58:41.154 | INFO     | autolgbm.autolgbm:_determine_problem_type:140 - Problem type: binary_classification
2022-03-16 00:58:41.193 | INFO     | autolgbm.utils:reduce_memory_usage:48 - Mem. usage decreased to 0.27 Mb (76.0% reduction)
2022-03-16 00:58:41.194 | INFO     | autolgbm.autolgbm:_create_folds:58 - Creating folds
2022-03-16 00:58:41.231 | INFO     | autolgbm.autolgbm:_process_data:170 - Encoding target(s)
2022-03-16 00:58:41.246 | INFO     | autolgbm.autolgbm:_process_data:195 - Found 0 categorical features.
2022-03-16 00:58:42.099 | INFO     | autolgbm.autolgbm:_process_data:236 - Model config: train_filename='./train_#13-book2_stg2.csv' test_filename='./test_#13-book2_stg2.csv' idx='id' targets=['target'] problem_type=<ProblemType.binary_cla

[LightGBM] [Warning] lambda_l1 is set=0.0004615770188918128, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0004615770188918128
[LightGBM] [Warning] bagging_fraction is set=0.5631347822741353, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5631347822741353
[LightGBM] [Warning] feature_fraction is set=0.9717667583456363, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9717667583456363
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=2.606150244545689e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.606150244545689e-06


/home/dev/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dev/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dev/.local/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/dev/.local/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose

[LightGBM] [Warning] lambda_l1 is set=0.0004615770188918128, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0004615770188918128
[LightGBM] [Warning] bagging_fraction is set=0.5631347822741353, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5631347822741353
[LightGBM] [Warning] feature_fraction is set=0.9717667583456363, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9717667583456363
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=2.606150244545689e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.606150244545689e-06


/home/dev/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dev/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dev/.local/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/dev/.local/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose

...

2022-03-16 03:01:29.886 | INFO     | autolgbm.utils:predict_model:331 - Fold 7 done!
2022-03-16 03:01:29.887 | INFO     | autolgbm.utils:predict_model:237 - Training and predicting for fold 8
/home/dev/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dev/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dev/.local/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'

[LightGBM] [Warning] lambda_l1 is set=1.8926330410437173, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8926330410437173
[LightGBM] [Warning] bagging_fraction is set=0.6890423907511551, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6890423907511551
[LightGBM] [Warning] feature_fraction is set=0.9730410606262032, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9730410606262032
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] lambda_l2 is set=3.6062621580193275, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.6062621580193275


2022-03-16 03:01:38.667 | INFO     | autolgbm.utils:predict_model:331 - Fold 8 done!
2022-03-16 03:01:38.667 | INFO     | autolgbm.utils:predict_model:237 - Training and predicting for fold 9
/home/dev/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dev/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dev/.local/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'

[LightGBM] [Warning] lambda_l1 is set=1.8926330410437173, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8926330410437173
[LightGBM] [Warning] bagging_fraction is set=0.6890423907511551, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6890423907511551
[LightGBM] [Warning] feature_fraction is set=0.9730410606262032, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9730410606262032
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] lambda_l2 is set=3.6062621580193275, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.6062621580193275


2022-03-16 03:01:46.994 | INFO     | autolgbm.utils:predict_model:331 - Fold 9 done!
2022-03-16 03:01:46.995 | INFO     | autolgbm.utils:predict_model:334 - Metrics: {'auc': 0.8430289513115673, 'binary_logloss': 0.48580957466924773, 'f1': 0.7581099961200156, 'accuracy': 0.7580246994501263, 'precision': 0.7578796906128782, 'recall': 0.7583831851659765}


# Inference

**魔理沙:最後にsubmissionファイルを作ろう！**

**霊夢:最後まで頑張るぞ！**

<br>

**Marisa: Finally, let's create a submission file!** <br>

**Reimu: I'll do my best until the end!**

<img src = "https://4.bp.blogspot.com/-mg-ul7z64Oc/WzC-BvpjsgI/AAAAAAABM94/-OPzexySWfUXUdf6qzZ45CfERCUyoNpBQCLcBGAs/s400/yaruki_moeru_woman.png" width=200>

In [126]:
# submission = pd.read_csv(BASE_PATH + "MSampleSubmissionStage1.csv") # for Stage 1
# submission = pd.read_csv(BASE_PATH + "MSampleSubmissionStage2.csv") # for Stage 2
SAMPLEFILE = "MSampleSubmissionStage2.csv" if FOR_STAGE2 else "MSampleSubmissionStage1.csv"
SUBMISSION_FILE = "./output/#13-book2_stg2_submission.csv" if FOR_STAGE2 else "./output/#13-book2_submission.csv"
submission = pd.read_csv(BASE_PATH + SAMPLEFILE)
autolgb_pred = pd.read_csv(output + "/test_predictions.csv")
submission['Pred'] = autolgb_pred['1.0']
# submission.rename(columns = {'1.0':'Pred'}, inplace = True)
submission.to_csv(SUBMISSION_FILE, index=False)
submission.head()

,ID,Pred
0,2022_1103_1104,0.078337
1,2022_1103_1112,0.029792
2,2022_1103_1116,0.096392
3,2022_1103_1120,0.045894
4,2022_1103_1124,0.044595


**霊夢:どんな特徴量を作ったら精度が上がるのかな？**

**魔理沙:それはたくさんの人のcodeを見て学ばないとね。**

**霊夢:もっと頑張らなくちゃだね。**

<br>

**Reimu: What kind of features should I make to improve the accuracy?**  

**Marisa: It has to be learned by looking at the codes of many people.**  

**Reimu: I have to work harder.**

***

**霊夢&魔理沙:ご覧いただきありがとうございました！良ければUpvoteをよろしくお願いします！**<br>

**Reimu & Marisa: Thank you for visiting! If you like, thank you for Upvote!**

![](https://1.bp.blogspot.com/-PNcKwFw1PpM/U1T3oDIr9CI/AAAAAAAAfT4/gEn86X8Ppx0/s400/figure_goodjob.png)